In [ ]:
import datetime
import numpy as np
import pandas as pd
import re
import plotly.express as px
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

In [ ]:
contacts_df = pd.read_excel('/content/Contacts (Done).xlsx', dtype={'Id': str})
calls_df = pd.read_excel('/content/Calls (Done).xlsx', dtype={'Id': str, "CONTACTID": str})
spend_df = pd.read_excel('/content/Spend (Done).xlsx')
deals_df = pd.read_excel('/content/Deals (Done).xlsx', dtype={'Id': str, 'Contact Name': str})

# 1. Очистка и подготовка данных

## `contacts_df`

In [ ]:
contacts_df.head()

Id Contact Owner Name      Created Time     Modified Time
0  5805028000000645014       Rachel White  27.06.2023 11:28  22.12.2023 13:34
1  5805028000000872003      Charlie Davis  03.07.2023 11:31  21.05.2024 10:23
2  5805028000000889001          Bob Brown  02.07.2023 22:37  21.12.2023 13:17
3  5805028000000907006          Bob Brown  03.07.2023 05:44  29.12.2023 15:20
4  5805028000000939010         Nina Scott  04.07.2023 10:11  16.04.2024 16:14

In [ ]:
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  object
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: object(4)
memory usage: 579.8+ KB


In [ ]:
contacts_df.describe()

Id Contact Owner Name      Created Time  \
count                 18548              18548             18548   
unique                18548                 28             17921   
top     5805028000056907001      Charlie Davis  10.06.2024 09:00   
freq                      1               2018                13   

           Modified Time  
count              18548  
unique             16580  
top     13.06.2024 17:08  
freq                  25

### Преобразования типов данных

In [ ]:
contacts_df["Created Time"] = pd.to_datetime(contacts_df["Created Time"], errors="raise")
contacts_df["Modified Time"] = pd.to_datetime(contacts_df["Modified Time"], errors="raise")

### Удаление дубликатов

In [ ]:
contacts_df[contacts_df.duplicated(keep=False, subset=contacts_df.columns[1:])]

Id Contact Owner Name        Created Time  \
203    5805028000001949074          Bob Brown 2023-07-17 18:40:00   
205    5805028000001953081          Bob Brown 2023-07-17 18:40:00   
279    5805028000002340967          Bob Brown 2023-07-18 16:53:00   
280    5805028000002344049          Bob Brown 2023-07-18 16:53:00   
334    5805028000002740077          Bob Brown 2023-07-21 12:26:00   
...                    ...                ...                 ...   
17864  5805028000054231884       Rachel White 2024-06-10 09:00:00   
17865  5805028000054232018       Rachel White 2024-06-10 09:00:00   
17869  5805028000054238271       Rachel White 2024-06-10 09:00:00   
17870  5805028000054238317       Rachel White 2024-06-10 09:00:00   
17872  5805028000054238365       Rachel White 2024-06-10 09:00:00   

            Modified Time  
203   2023-07-17 18:40:00  
205   2023-07-17 18:40:00  
279   2023-07-18 16:53:00  
280   2023-07-18 16:53:00  
334   2023-07-21 12:26:00  
...                   ...  
17864 2024-06-10 09:33:00  
17865 2024-06-10 09:33:00  
17869 2024-06-10 09:33:00  
17870 2024-06-10 09:33:00  
17872 2024-06-10 09:33:00  

[71 rows x 4 columns]

In [ ]:
contacts_df[contacts_df.duplicated(subset=contacts_df.columns[1:])].shape

(38, 4)

In [ ]:
contacts_df.drop_duplicates(subset=contacts_df.columns[1:], inplace=True)

contacts_df[contacts_df.duplicated(subset=contacts_df.columns[1:])].shape

(0, 4)

## Нашёлся один False в столбце Contact Owner Name

In [ ]:
contacts_df = contacts_df[contacts_df['Contact Owner Name'] != False]

In [ ]:
contacts_df[contacts_df['Contact Owner Name'] == False]

Empty DataFrame
Columns: [Id, Contact Owner Name, Created Time, Modified Time]
Index: []

### Обработка пропусков

In [ ]:
contacts_df.isna().sum()

Id                    0
Contact Owner Name    0
Created Time          0
Modified Time         0
dtype: int64

### Вывод итогов

In [ ]:
contacts_df.head()

Id Contact Owner Name        Created Time  \
0  5805028000000645014       Rachel White 2023-06-27 11:28:00   
1  5805028000000872003      Charlie Davis 2023-07-03 11:31:00   
2  5805028000000889001          Bob Brown 2023-07-02 22:37:00   
3  5805028000000907006          Bob Brown 2023-07-03 05:44:00   
4  5805028000000939010         Nina Scott 2023-07-04 10:11:00   

        Modified Time  
0 2023-12-22 13:34:00  
1 2024-05-21 10:23:00  
2 2023-12-21 13:17:00  
3 2023-12-29 15:20:00  
4 2024-04-16 16:14:00

In [ ]:
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18509 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18509 non-null  object        
 1   Contact Owner Name  18509 non-null  object        
 2   Created Time        18509 non-null  datetime64[ns]
 3   Modified Time       18509 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 723.0+ KB


In [ ]:
contacts_df.describe()

Created Time                  Modified Time
count                          18509                          18509
mean   2024-01-24 14:35:26.704846336  2024-02-15 09:20:47.305634816
min              2023-06-27 11:28:00            2023-07-06 10:54:00
25%              2023-11-15 17:08:00            2023-12-09 14:54:00
50%              2024-02-01 18:45:00            2024-02-29 01:35:00
75%              2024-04-12 16:20:00            2024-04-26 22:48:00
max              2024-06-21 15:30:00            2024-06-21 15:32:00

In [ ]:
contacts_df.to_excel("Contacts_clear.xlsx")

In [ ]:
contacts_df.to_csv('Contacts_clear.csv', index=False)

In [ ]:
files.download('Contacts_clear.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# `calls_df`

In [ ]:
calls_df.head()

Id   Call Start Time Call Owner Name            CONTACTID  \
0  5805028000000805001  30.06.2023 08:43        John Doe                  NaN   
1  5805028000000768006  30.06.2023 08:46        John Doe                  NaN   
2  5805028000000764027  30.06.2023 08:59        John Doe                  NaN   
3  5805028000000787003  30.06.2023 09:20        John Doe  5805028000000645014   
4  5805028000000768019  30.06.2023 09:30        John Doe  5805028000000645014   

  Call Type  Call Duration (in seconds)       Call Status  Dialled Number  \
0   Inbound                       171.0          Received             NaN   
1  Outbound                        28.0  Attended Dialled             NaN   
2  Outbound                        24.0  Attended Dialled             NaN   
3  Outbound                         6.0  Attended Dialled             NaN   
4  Outbound                        11.0  Attended Dialled             NaN   

  Outgoing Call Status  Scheduled in CRM  Tag  
0                  NaN               NaN  NaN  
1            Completed               0.0  NaN  
2            Completed               0.0  NaN  
3            Completed               0.0  NaN  
4            Completed               0.0  NaN

In [ ]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  object 
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  object 
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                         0 non-null      float64
dtypes: float64(4), object(7)
memory usage: 8.0+ MB


In [ ]:
calls_df.describe()

Call Duration (in seconds)  Dialled Number  Scheduled in CRM  Tag
count                95791.000000             0.0      86875.000000  0.0
mean                   164.977263             NaN          0.001635  NaN
std                    401.410826             NaN          0.040397  NaN
min                      0.000000             NaN          0.000000  NaN
25%                      4.000000             NaN          0.000000  NaN
50%                      8.000000             NaN          0.000000  NaN
75%                     98.000000             NaN          0.000000  NaN
max                   7625.000000             NaN          1.000000  NaN

### Преобразования типо данных

In [ ]:
calls_df["Call Start Time"] = pd.to_datetime(calls_df["Call Start Time"], errors="raise")
calls_df['Call Duration (in seconds)'] = calls_df['Call Duration (in seconds)'].fillna(0).astype(int)

In [ ]:
calls_df['Call Duration (min)'] = calls_df['Call Duration (in seconds)'] / 60

In [ ]:
calls_df[calls_df['Call Duration (min)'] > 0]

Id     Call Start Time Call Owner Name  \
0      5805028000000805001 2023-06-30 08:43:00        John Doe   
1      5805028000000768006 2023-06-30 08:46:00        John Doe   
2      5805028000000764027 2023-06-30 08:59:00        John Doe   
3      5805028000000787003 2023-06-30 09:20:00        John Doe   
4      5805028000000768019 2023-06-30 09:30:00        John Doe   
...                    ...                 ...             ...   
95866  5805028000056872334 2024-06-21 15:26:00    Kevin Parker   
95867  5805028000056839488 2024-06-21 15:29:00   Ulysses Adams   
95869  5805028000056889515 2024-06-21 15:30:00   Ulysses Adams   
95870  5805028000056875317 2024-06-21 15:30:00   Victor Barnes   
95871  5805028000056832495 2024-06-21 15:30:00    Kevin Parker   

                 CONTACTID Call Type  Call Duration (in seconds)  \
0                      NaN   Inbound                         171   
1                      NaN  Outbound                          28   
2                      NaN  Outbound                          24   
3      5805028000000645014  Outbound                           6   
4      5805028000000645014  Outbound                          11   
...                    ...       ...                         ...   
95866  5805028000025817526  Outbound                          54   
95867  5805028000056564231  Outbound                           5   
95869  5805028000056564231  Outbound                           6   
95870  5805028000054867023  Outbound                           8   
95871  5805028000010617278  Outbound                           5   

            Call Status  Dialled Number Outgoing Call Status  \
0              Received             NaN                  NaN   
1      Attended Dialled             NaN            Completed   
2      Attended Dialled             NaN            Completed   
3      Attended Dialled             NaN            Completed   
4      Attended Dialled             NaN            Completed   
...                 ...             ...                  ...   
95866  Attended Dialled             NaN            Completed   
95867  Attended Dialled             NaN            Completed   
95869  Attended Dialled             NaN            Completed   
95870  Attended Dialled             NaN            Completed   
95871  Attended Dialled             NaN            Completed   

       Scheduled in CRM  Tag  Call Duration (min)  
0                   NaN  NaN             2.850000  
1                   0.0  NaN             0.466667  
2                   0.0  NaN             0.400000  
3                   0.0  NaN             0.100000  
4                   0.0  NaN             0.183333  
...                 ...  ...                  ...  
95866               0.0  NaN             0.900000  
95867               0.0  NaN             0.083333  
95869               0.0  NaN             0.100000  
95870               0.0  NaN             0.133333  
95871               0.0  NaN             0.083333  

[73816 rows x 12 columns]

### Удаление дубликатов и столбцов

In [ ]:
duplicate = calls_df.duplicated(subset=calls_df.columns[1:]) & (calls_df['Call Duration (min)'] > 0)
calls_df = calls_df[~duplicate]

In [ ]:
calls_df[calls_df.duplicated(subset=calls_df.columns[1:])].shape

(2067, 12)

In [ ]:
calls_df.drop_duplicates(subset=calls_df.columns[1:], inplace=True)

In [ ]:
calls_df[calls_df.duplicated(subset=calls_df.columns[1:])].shape

(0, 12)

In [ ]:
calls_df = calls_df.dropna(subset=['CONTACTID'])

In [ ]:
colum_to_drop = ['Dialled Number', 'Tag']
calls_df = calls_df.drop(columns=colum_to_drop)

## Обработка пропусков

In [ ]:
columns_1 = ['Outgoing Call Status','Scheduled in CRM']
calls_df[columns_1] = calls_df[columns_1].fillna("Unknown")

In [ ]:
calls_df.isna().sum()

Id                            0
Call Start Time               0
Call Owner Name               0
CONTACTID                     0
Call Type                     0
Call Duration (in seconds)    0
Call Status                   0
Outgoing Call Status          0
Scheduled in CRM              0
Call Duration (min)           0
dtype: int64

### Вывод итогов

In [ ]:
calls_df.head()

Id     Call Start Time Call Owner Name  \
3   5805028000000787003 2023-06-30 09:20:00        John Doe   
4   5805028000000768019 2023-06-30 09:30:00        John Doe   
5   5805028000000790004 2023-06-30 12:09:00        John Doe   
6   5805028000000773022 2023-06-30 14:24:00        John Doe   
10  5805028000000942048 2023-07-04 15:35:00      Jane Smith   

              CONTACTID Call Type  Call Duration (in seconds)  \
3   5805028000000645014  Outbound                           6   
4   5805028000000645014  Outbound                          11   
5   5805028000000645014  Outbound                          12   
6   5805028000000645014  Outbound                           4   
10  5805028000000645014  Outbound                          20   

         Call Status Outgoing Call Status Scheduled in CRM  \
3   Attended Dialled            Completed              0.0   
4   Attended Dialled            Completed              0.0   
5   Attended Dialled            Completed              0.0   
6   Attended Dialled            Completed              0.0   
10  Attended Dialled            Completed              0.0   

    Call Duration (min)  
3              0.100000  
4              0.183333  
5              0.200000  
6              0.066667  
10             0.333333

In [ ]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88815 entries, 3 to 95872
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          88815 non-null  object        
 1   Call Start Time             88815 non-null  datetime64[ns]
 2   Call Owner Name             88815 non-null  object        
 3   CONTACTID                   88815 non-null  object        
 4   Call Type                   88815 non-null  object        
 5   Call Duration (in seconds)  88815 non-null  int64         
 6   Call Status                 88815 non-null  object        
 7   Outgoing Call Status        88815 non-null  object        
 8   Scheduled in CRM            88815 non-null  object        
 9   Call Duration (min)         88815 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 7.5+ MB


In [ ]:
calls_df.describe()

Call Start Time  Call Duration (in seconds)  \
count                          88815                88815.000000   
mean   2024-02-04 12:52:24.652592384                  171.030108   
min              2023-06-30 09:20:00                    0.000000   
25%              2023-11-23 12:45:00                    4.000000   
50%              2024-02-16 13:23:00                    9.000000   
75%              2024-04-22 12:51:00                  108.000000   
max              2024-06-21 15:30:00                 7625.000000   
std                              NaN                  408.379382   

       Call Duration (min)  
count         88815.000000  
mean              2.850502  
min               0.000000  
25%               0.066667  
50%               0.150000  
75%               1.800000  
max             127.083333  
std               6.806323

In [ ]:
calls_df.to_excel("Calls_clear.xlsx")

In [ ]:
calls_df.to_csv('Calls_clear.csv', index=False)

In [ ]:
files.download('Calls_clear.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# `spend_df`

In [ ]:
spend_df.head()

Date        Source               Campaign  Impressions  Spend  Clicks  \
0 2023-07-03    Google Ads         gen_analyst_DE            6   0.00       0   
1 2023-07-03    Google Ads  performancemax_eng_DE            4   0.01       1   
2 2023-07-03  Facebook Ads                    NaN            0   0.00       0   
3 2023-07-03    Google Ads                    NaN            0   0.00       0   
4 2023-07-03           CRM                    NaN            0   0.00       0   

  AdGroup   Ad  
0     NaN  NaN  
1     NaN  NaN  
2     NaN  NaN  
3     NaN  NaN  
4     NaN  NaN

In [ ]:
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetime64[ns]
 1   Source       20779 non-null  object        
 2   Campaign     14785 non-null  object        
 3   Impressions  20779 non-null  int64         
 4   Spend        20779 non-null  float64       
 5   Clicks       20779 non-null  int64         
 6   AdGroup      13951 non-null  object        
 7   Ad           13951 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.3+ MB


In [ ]:
spend_df.describe()

Date    Impressions         Spend  \
count                          20779   20779.000000  20779.000000   
mean   2024-01-14 22:32:40.864334080    2458.203475      7.195892   
min              2023-07-03 00:00:00       0.000000      0.000000   
25%              2023-10-13 00:00:00       0.000000      0.000000   
50%              2024-01-27 00:00:00      63.000000      0.580000   
75%              2024-04-16 00:00:00     709.000000      5.750000   
max              2024-06-21 00:00:00  431445.000000    774.000000   
std                              NaN   11442.528075     26.760080   

             Clicks  
count  20779.000000  
mean      23.990616  
min        0.000000  
25%        0.000000  
50%        1.000000  
75%       12.000000  
max     2415.000000  
std       85.245714

### Преобразования типо данных

In [ ]:
spend_df["Date"] = pd.to_datetime(spend_df["Date"], errors="raise")

### Удаление дубликатов

In [ ]:
spend_dup = spend_df[spend_df.duplicated(subset=spend_df.columns[1:])]

In [ ]:
spend_df.drop_duplicates(subset=spend_df.columns[1:], inplace=True)

In [ ]:
spend_df[spend_df.duplicated(subset=spend_df.columns[1:])].shape

(0, 8)

### Обработка пропусков

In [ ]:
columns_1 = ['Campaign', 'AdGroup', 'Ad']
spend_df[columns_1] = spend_df[columns_1].fillna("Unknown")

In [ ]:
spend_df = spend_df[~spend_df['Source'].str.contains('Test', case=False, na=False)]

In [ ]:
spend_df.isna().sum()

Date           0
Source         0
Campaign       0
Impressions    0
Spend          0
Clicks         0
AdGroup        0
Ad             0
dtype: int64

### Вывод итогов

In [ ]:
spend_df.head()

Date        Source               Campaign  Impressions  Spend  Clicks  \
0 2023-07-03    Google Ads         gen_analyst_DE            6   0.00       0   
1 2023-07-03    Google Ads  performancemax_eng_DE            4   0.01       1   
2 2023-07-03  Facebook Ads                Unknown            0   0.00       0   
3 2023-07-03    Google Ads                Unknown            0   0.00       0   
4 2023-07-03           CRM                Unknown            0   0.00       0   

   AdGroup       Ad  
0  Unknown  Unknown  
1  Unknown  Unknown  
2  Unknown  Unknown  
3  Unknown  Unknown  
4  Unknown  Unknown

In [ ]:
spend_df.describe()

Date    Impressions         Spend  \
count                          14887   14887.000000  14887.000000   
mean   2024-01-09 17:16:55.893061376    3427.332774      9.983453   
min              2023-07-03 00:00:00       0.000000      0.000000   
25%              2023-10-05 00:00:00      34.000000      0.300000   
50%              2024-01-22 00:00:00     286.000000      2.330000   
75%              2024-04-11 00:00:00    1060.000000      8.885000   
max              2024-06-21 00:00:00  431445.000000    774.000000   
std                              NaN   13395.468506     31.159348   

             Clicks  
count  14887.000000  
mean      31.062806  
min        0.000000  
25%        0.000000  
50%        4.000000  
75%       17.000000  
max     2415.000000  
std       98.414714

In [ ]:
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14887 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         14887 non-null  datetime64[ns]
 1   Source       14887 non-null  object        
 2   Campaign     14887 non-null  object        
 3   Impressions  14887 non-null  int64         
 4   Spend        14887 non-null  float64       
 5   Clicks       14887 non-null  int64         
 6   AdGroup      14887 non-null  object        
 7   Ad           14887 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.0+ MB


In [ ]:
spend_df.to_excel("Spend_clear.xlsx")

In [ ]:
spend_df.to_csv('Spend_clear.csv', index=False)

In [ ]:
files.download('Spend_clear.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# `deals_df`

In [ ]:
deals_df.head()

Id Deal Owner Name Closing Date            Quality  \
0  5805028000056864695        Ben Hall          NaN                NaN   
1  5805028000056859489   Ulysses Adams          NaN                NaN   
2  5805028000056832357   Ulysses Adams   21.06.2024     D - Non Target   
3  5805028000056824246        Eva Kent   21.06.2024  E - Non Qualified   
4  5805028000056873292        Ben Hall   21.06.2024     D - Non Target   

      Stage     Lost Reason       Page                  Campaign       SLA  \
0  New Lead             NaN  /eng/test             03.07.23women       NaN   
1  New Lead             NaN    /at-eng                       NaN       NaN   
2      Lost      Non target    /at-eng                engwien_AT  00:26:43   
3      Lost  Invalid number       /eng  04.07.23recentlymoved_DE  01:00:04   
4      Lost      Non target       /eng              discovery_DE  00:53:12   

              Content  ...        Product Education Type      Created Time  \
0                 v16  ...            NaN            NaN  21.06.2024 15:30   
1                 NaN  ...  Web Developer        Morning  21.06.2024 15:23   
2               b1-at  ...            NaN            NaN  21.06.2024 14:45   
3  bloggersvideo14com  ...            NaN            NaN  21.06.2024 13:32   
4             website  ...            NaN            NaN  21.06.2024 13:21   

  Course duration Months of study Initial Amount Paid  Offer Total Amount  \
0             NaN             NaN                 NaN                 NaN   
1             6.0             NaN                   0                2000   
2             NaN             NaN                 NaN                 NaN   
3             NaN             NaN                 NaN                 NaN   
4             NaN             NaN                 NaN                 NaN   

          Contact Name City Level of Deutsch  
0  5805028000056849495  NaN              NaN  
1  5805028000056834471  NaN              NaN  
2  5805028000056854421  NaN              NaN  
3  5805028000056889351  NaN              NaN  
4  5805028000056876176  NaN              NaN  

[5 rows x 23 columns]

In [ ]:
deals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21595 entries, 0 to 21594
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   21593 non-null  object 
 1   Deal Owner Name      21564 non-null  object 
 2   Closing Date         14645 non-null  object 
 3   Quality              19340 non-null  object 
 4   Stage                21593 non-null  object 
 5   Lost Reason          16124 non-null  object 
 6   Page                 21593 non-null  object 
 7   Campaign             16067 non-null  object 
 8   SLA                  15533 non-null  object 
 9   Content              14147 non-null  object 
 10  Term                 12454 non-null  object 
 11  Source               21593 non-null  object 
 12  Payment Type         496 non-null    object 
 13  Product              3592 non-null   object 
 14  Education Type       3300 non-null   object 
 15  Created Time         21593 non-null 

In [ ]:
deals_df.describe()

Course duration  Months of study
count      3587.000000       840.000000
mean         10.198495         5.442857
std           1.834681         2.919920
min           6.000000         0.000000
25%          11.000000         3.000000
50%          11.000000         5.000000
75%          11.000000         8.000000
max          11.000000        11.000000

## Преобразования типо данных

In [ ]:
deals_df["Created Time"] = pd.to_datetime(deals_df["Created Time"], errors="raise")
deals_df["Closing Date"] = pd.to_datetime(deals_df["Closing Date"], errors="raise")

## В столбце SLA время приведем к единому виду - секунды

In [ ]:
deals_df['SLA'].value_counts()

SLA
00:10:11    6
00:10:46    6
00:11:09    6
00:15:37    5
00:17:43    5
           ..
00:03:22    1
03:52:09    1
00:04:47    1
00:00:27    1
00:06:48    1
Name: count, Length: 13357, dtype: int64

In [ ]:
deals_df["SLA"].apply(type).value_counts()

SLA
<class 'datetime.time'>         13672
<class 'float'>                  6062
<class 'datetime.timedelta'>     1861
Name: count, dtype: int64

In [ ]:
deals_df["SLA"].isna().sum()

np.int64(6062)

In [ ]:
deals_df["SLA"][2]

datetime.time(0, 26, 43)

In [ ]:
def convert_to_seconds(x):
    if pd.isna(x):
        return np.nan
    elif isinstance(x, datetime.time):
        return x.hour * 3600 + x.minute * 60 + x.second
    elif isinstance(x, datetime.timedelta):
        return x.total_seconds()

deals_df['SLA Seconds'] = deals_df["SLA"].apply(convert_to_seconds)
deals_df['SLA Seconds'].value_counts()

SLA Seconds
611.0      6
646.0      6
669.0      6
937.0      5
1063.0     5
          ..
202.0      1
13929.0    1
287.0      1
27.0       1
408.0      1
Name: count, Length: 13357, dtype: int64

In [ ]:
deals_df.drop(columns=['SLA'], inplace=True)

## Обработка пропусков

In [ ]:
deals_df.isna().sum()

Id                         2
Deal Owner Name           31
Closing Date            6950
Quality                 2255
Stage                      2
Lost Reason             5471
Page                       2
Campaign                5528
Content                 7448
Term                    9141
Source                     2
Payment Type           21099
Product                18003
Education Type         18295
Created Time               2
Course duration        18008
Months of study        20755
Initial Amount Paid    17430
Offer Total Amount     17410
Contact Name              63
City                   19084
Level of Deutsch       20344
SLA Seconds             6062
dtype: int64

In [ ]:
deals_df.drop_duplicates(subset=deals_df.columns[1:], inplace=True)

In [ ]:
deals_df = deals_df.dropna(subset=['Contact Name'])

In [ ]:
deals_df = deals_df.drop(columns=['Page'])

In [ ]:
deals_df = deals_df[~deals_df['Lost Reason'].str.contains('Duplicate', case=False, na=False)]

In [ ]:
columns = ['Deal Owner Name', 'Quality', 'Lost Reason', 'Campaign', 'Content', 'Term']
deals_df[columns] = deals_df[columns].fillna("Unknown")

In [ ]:
deals_df = deals_df[~deals_df['Source'].str.contains('Test', case=False, na=False)]

In [ ]:
deals_df.drop_duplicates(subset=deals_df.columns[1:], inplace=True)

In [ ]:
deals_df.isna().sum()

Id                         0
Deal Owner Name            0
Closing Date            6583
Quality                    0
Stage                      0
Lost Reason                0
Campaign                   0
Content                    0
Term                       0
Source                     0
Payment Type           19137
Product                16120
Education Type         16399
Created Time               0
Course duration        16124
Months of study        18787
Initial Amount Paid    15594
Offer Total Amount     15574
Contact Name               0
City                   17138
Level of Deutsch       18390
SLA Seconds             4800
dtype: int64

In [ ]:
mode_values = deals_df.groupby('Contact Name')['City'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
deals_df['City'] = deals_df['Contact Name'].map(mode_values)

In [ ]:
mode_values

Contact Name
5805028000000872003       None
5805028000000907006       None
5805028000000939010    Hamburg
5805028000000942003       None
5805028000000961001       None
                        ...   
5805028000056889019       None
5805028000056889209       None
5805028000056889351       None
5805028000056892018       None
5805028000056892055       None
Name: City, Length: 16869, dtype: object

In [ ]:
mode_values = deals_df.groupby('Contact Name')['Level of Deutsch'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
deals_df['Level of Deutsch'] = deals_df['Contact Name'].map(mode_values)

## Проверка столбцов открытия и закрытия заявки

In [ ]:
swapped_dates = deals_df['Closing Date'] < deals_df['Created Time']
deals_df.loc[swapped_dates, ['Created Time', 'Closing Date']] = deals_df.loc[swapped_dates, ['Closing Date', 'Created Time']].values
count_swapped = swapped_dates.sum()

print(f"Количество исправленных строк: {count_swapped}")

Количество исправленных строк: 2380


In [ ]:
deals_df['Difference in Days'] = (deals_df['Closing Date'] - deals_df['Created Time']).dt.days

## Приводим столбец Level of Deutsch к одному виду







In [ ]:
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch']

In [ ]:
deals_df['Level of Deutsch_1'].count()
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].str.lower()
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].str.replace(r'а(?=[12])', 'a', regex=True)
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].str.replace(r'с(?=[12])', 'c', regex=True)
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].str.replace(r'[вб](?=1)', 'b', regex=True)
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].str.replace(r'[вб](?=2)', 'b', regex=True)
priority = {'a1': 1, 'a2': 2, 'b1': 3, 'b2': 4, 'c1': 5, 'c2': 6}
def extract_highest_level(text):
    if pd.isna(text):
        return np.nan
    matches = re.findall(r'[a-c][1-2]', text)
    if matches:
        return max(matches, key=lambda x: priority[x])
    else:
        return 'a1'
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].apply(extract_highest_level)
deals_df[deals_df['Level of Deutsch'].notna()][['Level of Deutsch_1', 'Level of Deutsch']].sample(3)
deals_df.drop(columns=['Level of Deutsch'], inplace=True)

In [ ]:
replace_dict = {
    'в1': 'B1', 'б1': 'B1', 'b1': 'B1', 'B1': 'B1', 'в1-в2': 'B1-B2', 'B2': 'B2',
    'C2': 'C2', 'с1': 'C1', 'Б1': 'B1', 'а2': 'A2', 'а1': 'A1', 'а0': 'A0',
    'б2': 'B2', 'Б2': 'B2', 'В1': 'B1', 'А2': 'A2', 'B1 будет в феврале 2025': 'B1',
    'Detmold, Paulinenstraße 95, 32756': 'Неизвестно', 'Сам оценивает на B2, 13 лет живет в Германии': 'B2',
    'в2': 'B2', 'В1-В2': 'B1-B2', 'Б1 ( ждет Б2)': 'B1-B2', 'А2-В1': 'A2-B1',
    'lэкзамен - 6 июля на В1. курсы вечером (но уверенно говорит на B1)': 'B1',
    'Гражданка Германии уже год в Германии Учит немецкий и в сентябре b1 через гос-во проходит, а не через ДЖЦ, вечером учится 3 р в неделю с 18 до 21': 'B1',
    '-': 'A0', 'А2 ( Б1 в июне)': 'A2-B1', 'B1 в процессе обучения': 'B1',
    'ЯЗ: нем В1 был экз 03.05 повтор и сейчас ждет результаты. Технический англ был. А1 сейчас. ОБР: 2 во информационные и комп сети - инженер системоте': 'B1',
    'В1 в сентябре': 'B1', 'Нет': 'Неизвестно', 'С1': 'C1', 0: 'A0',
    'Ждем B1': 'B1', 'А1 сертиф, но по факту А2': 'A2', 'a2': 'A2', 'Пока А2, сдает 17 05 B1': 'A2-B1',
    'окончание 13.06 курса на b1': 'B1', 'A1': 'A1', 'b2': 'B2', 'Thorn-Prikker-Str. 30, Hagen, 58093': 'Неизвестно',
    'В2': 'B2', 'нулевой уровень, только пошел на курсы.': 'A0', 'ая в1': 'B1',
    'Ждет результат по B1': 'B1', 'А2( ждет итоги Б!)': 'A2-B1', 'b1 экзамен будет 12 апреля': 'B1',
    'b1 (b2 ждет серт)': 'B1-B2', 'С2': 'C2', 'ждем B1': 'B1', 'Paderborn 33102, Schwabenweg 10': 'Неизвестно',
    'b1 (B2 ждет серт)': 'B1-B2', 'Ждем B1 со дня на день': 'B1', 'Б2 ( учит С1)': 'B2-C1',
    'B1 еще нет результата': 'B1', '31.05.2024': 'Неизвестно', 'Lichtenfelser Straße 25, Untersiemau 96253': 'Неизвестно',
    'Учиться до сентября на B1': 'B1', 'b1 9ждет экзамен)': 'B1', 'b1+': 'B1', 'гражданка': 'B2',
    'b1 (ждет результат)': 'B1', 'Б1 (учит Б2)': 'B1-B2', 'б2+': 'B2', 'Гражданин': 'B2',
    '25 лет живет в Германии': 'C1', 'С1 -ая , Ня -а1': 'C1', 'Ждем результат по B1': 'B1',
    'b1 (b2 в июле экзамен)': 'B1-B2', 'Ждет со дня на день В1': 'B1',
    'А2 (В1 с 3 раза не сдала, бератер видела наши доки)': 'A2-B1', 'b1 (ждет результаты)': 'B1',
    'А2 ( повтор на Б1)': 'A2-B1', 'B1, сдает B2 в апреле': 'B1-B2', 'ждет сертификат B1': 'B1',
    'Б2( 16.02 экзамен С1)': 'B2-C1', 'А1-А2': 'A1-A2', 'b1 ждет серт на днях на руки': 'B1',
    'b1 24 февраля экзамен, англ b2': 'B1-B2', 'А2 ( скоро екзамен)': 'A2', 'B1 (ждет результаты В2)': 'B1-B2',
    'b1 (b2 15 марта экзамен)': 'B1-B2', 'b2 (с1 экзамен 16 февраля)': 'B2-C1', 'Б1 ( ждет итог Б2)': 'B1-B2',
    'не сдавал, но гражданин': 'B2', 'Нет сертификатов, но есть С1 англ, неоконченное высшее в ИТ (и еще одно высшее юридическое) , очень хочет в ИТ, сильно замотивирована именно н': 'Неизвестно',
    'А2, в процессе Б1': 'A2-B1', 'A0': 'A0', 'А2(Б1 в марте экз)': 'A2-B1', 'учит A2': 'A2',
    'Б1 ( проходит Б2)': 'B1-B2', 'Б1 ( ждет итог )': 'B1-B2', 'НЯ - В1, АЯ - В1': 'B1',
    'б1 (ждет рез-тат)': 'B1', 'А2(ждет итоги Б1)': 'A2-B1', 'в1-ня , в1-ая': 'B1',
    'ня-0, но англ B2+': 'A0', 'В': 'B1', 'будет B1 в июне': 'B1',
    'А2( включили нем в ангебот)': 'A2', 'а2-в1': 'A2-B1', 'в2-с1': 'B2-C1',
    'курс А2-В1 - сдача в июле, но вечерняя смена инт курсов, настроен получить гутшайн уже сейчас.': 'A2-B1',
    'B1 (B2 должна до конца февраля получить)': 'B1-B2', 'b1 (b2 экзамен 6 февраля)': 'B1-B2',
    'A1-A2': 'A1-A2', 'Б1( может будет)': 'B1', 'А2 ( в процессе Б1)': 'A2-B1',
    'b1 ждет результаты': 'B1', 'b1 ждет экзамен в феврале': 'B1', 'В1, может уже В2?': 'B1-B2',
    'A2 (идет доучивать В1 - 300 часов; предположительно до августа)': 'A2-B1', 'не учил': 'A0',
    'Без 5 минут B1 (ждет результаты экзамена)': 'B1', 'а1-а2 , ая свободный': 'A1-A2', 'b2-c2': 'B2',
    'а2, англ B1': 'A2', 'А1': 'A1', 'А2 нем -В2 англ': 'A2', 'Проходит сейчас B1': 'B1',
    'Ждет результат по B1 в феврале': 'B1', 'Проходит сейчас повторно B1': 'B1',
    'b1 экзамен в феврале': 'B1', 'Учиться на B1 во вторую смену, в первую хочет получить одобрение на обучение у нас': 'B1',
    'Б10Б2': 'B2', 'Б1?': 'B1', 'B1 есть, ждем B2 в конце месяца': 'B1-B2', 'B1-B2': 'B1-B2',
    '?': 'Неизвестно', 'b1 экзамен 26 января': 'B1', 'А0': 'A0', 'а2 (б1 в сер января)': 'A2-B1',
    'f2': 'A2', 'Учиться на B1': 'B1', 'Сдала экзамен на B1, ждет в начале февраля результат': 'B1',
    'Сдавал 8 12 на B1 - ждет результат. 3 01 - аплейт - получил B1!': 'B1', 'Б1-Б2': 'B1-B2',
    'б1 (до июля на В2)': 'B1-B2', 'А2 ( Б1 март )': 'A2-B1', 'А2 (весной - еще 300 часов В1)': 'A2-B1',
    'В январе будут результаты по экзамену на B1': 'B1', 'б2 (с1 ждет рез-тат)': 'B2-C1',
    'ня-0, ая-B1': 'A0', 'А2-Б1': 'A2-B1', 'B1 (почти, не сдала чуть) + англ В1': 'B1',
    'в1 ждем результаты': 'B1', 'А2 ( хочет просить совмещать)': 'A2', 'B1 (ждет результаты)': 'B1',
    'А2+': 'A2', 'а2 (сдавала экз В1, но не сдала похоже)': 'A2', 'в1, идет на в2': 'B1-B2',
    'b2-c1': 'B2-C1', 'C1': 'C1', 'b1-b2': 'B1-B2', 'не учила ( разговорный) сразу пошла работать': 'B2',
    'Б1 ( проходит Б2 )': 'B1-B2', 'a0-a1': 'A0', 'Б1 ( был екзамен ждет итог )': 'B1',
    'Б2-С1': 'B2-C1', 'b1 (учила, но не сдала В2)': 'B1', 'ня а2, ая в1': 'A2-B1',
    'A2 (идет на В1)': 'A2-B1', 'B2-C2': 'B2', 'немецкий - а1-а2, англ b1-b2': 'A2',
    'B2+': 'B2', 'в1, еще нет сертификата': 'B1', 'б1-б2': 'B1-B2', 'Бй': 'B1',
    'ждет результаты по B1 экзамену': 'B1', 'b2 (ждет серт)': 'B2', 'никакой': 'A0',
    'в1 , хочет совмещать с в2': 'B1-B2', 90: 'Неизвестно', '.': 'Неизвестно',
    'в1 (уже сдала В2)': 'B2', 'b1 результат экзамена в феврале': 'B1',
    'в1 , экзамен на в2 15 декабря': 'B1-B2', 'идет на А1': 'A1', 'УТОЧНИТЬ!': 'Неизвестно',
    'B2 (говорит без проблем - давно здесь)': 'B2', 'B1 (до февраля)': 'B1',
    'А2 ( Б2 в процессе)': 'B1', 'C': 'C1', 'б1 заканчивает': 'B1', 'B1 (B2 экзамен в январе)': 'B1-B2',
    '5 июля 2024 сдает экз на В2': 'B2', 'А2 (заканчив В1 в июне)': 'A2-B1',
    'a2-б1': 'A2-B1', 'В1?': 'B1', 'b1 будет в январе экзамен, готов совмещать': 'B1',
    'b1 (b2 экзамен 2 марта)': 'B1-B2', 'B1 немецкий и английский Advance': 'B1',
    'A': 'A1', 'a2 (b1 экзамен 15 июня)': 'A2-B1', 'B2 (ждет итог экзамена)': 'B2',
    'b1 (b2 не сдал экзамен)': 'B1', 'В1 (учится на В2 до авг.': 'B1-B2',
    'В2 - не сдал': 'B2', 'B2+ (не сдавал, но говорит)': 'B2', 'b1 (ждет серт)': 'B1',
    'B1 вроде был (18 лет назад сдавал)': 'B1', 'А2 (сдает B1 - 12 дек) - не сдал!': 'A2',
    'УТОЧНИТЬ': 'Неизвестно', 'b2 ждет серт': 'B2', 'разговорный из украины, без сертификата': 'B2',
    'Ждет B1': 'B1', 'сдавала А2 в сентябре': 'A2', 'В1, учится на В2 до няоб 24': 'B1-B2',
    'Б1 ( ждет результат Б2)': 'B1-B2', 'точно уровень не знаю, но говорить могу - учила сама': 'B2',
    'А2-В1 учит': 'A2-B1', 'В1 (учится на В2 уже)': 'B1', 'В январе - В2 сдает': 'B2',
    'b1 должна получить результаты в феврале': 'B1'
}
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].replace(replace_dict).fillna("Unknown")

In [ ]:
deals_df['Level of Deutsch_1'].unique()

array(['Unknown', 'B1', 'A2', 'B2', 'a1', 'c2', 'c1'], dtype=object)

In [ ]:
deals_df['Level of Deutsch_1'].value_counts()

Level of Deutsch_1
Unknown    18056
B1           975
B2           289
A2           155
a1            90
c1            43
c2             6
Name: count, dtype: int64

In [ ]:
replace_take_two = {"B1-B2": "B2", 'A2-B1': 'B1', 'B2-C1': 'C1', 'A1-A2': "A2"}
deals_df['Level of Deutsch_1'] = deals_df['Level of Deutsch_1'].replace(replace_take_two)

## Анализ столбцов Offer Total Amount и Initial Amount Paid



In [ ]:
deals_df[deals_df['Offer Total Amount'].notna()]

Id  Deal Owner Name        Closing Date  \
1      5805028000056859489    Ulysses Adams                 NaT   
56     5805028000056683030    Charlie Davis                 NaT   
60     5805028000056558351    Ulysses Adams                 NaT   
65     5805028000056555140  Paula Underwood                 NaT   
71     5805028000056564131         Ben Hall                 NaT   
...                    ...              ...                 ...   
21574  5805028000001020028       Jane Smith 2023-11-03 00:00:00   
21578  5805028000001040014   Quincy Vincent 2023-07-14 00:00:00   
21585  5805028000000935081     Julia Nelson                 NaT   
21586  5805028000000947046    Oliver Taylor                 NaT   
21592  5805028000000922001        Bob Brown 2023-07-03 17:03:00   

                 Quality                Stage                  Lost Reason  \
1                Unknown             New Lead                      Unknown   
56               C - Low  Waiting For Payment                      Unknown   
60               C - Low  Waiting For Payment                      Unknown   
65            B - Medium  Waiting For Payment                      Unknown   
71        D - Non Target  Waiting For Payment                      Unknown   
...                  ...                  ...                          ...   
21574            C - Low                 Lost             Changed Decision   
21578            C - Low                 Lost  Conditions are not suitable   
21585     D - Non Target         Call Delayed                      Unknown   
21586         B - Medium         Payment Done                      Unknown   
21592  E - Non Qualified                 Lost                      Refugee   

                                Campaign             Content           Term  \
1                                Unknown             Unknown        Unknown   
56                 performancemax_eng_DE     _{region_name}_        Unknown   
60                               Unknown             Unknown        Unknown   
65                         03.07.23women                 v16          women   
71                  20.05.24interests_DE  bloggersvideo14com  interest_work   
...                                  ...                 ...            ...   
21574  performancemax_digitalmarkt_ru_DE     _{region_name}_        Unknown   
21578                    02.07.23wide_DE                  b3           wide   
21585                      03.07.23women                  b3          women   
21586                    02.07.23wide_DE                  b3           wide   
21592                            Unknown             Unknown        Unknown   

             Source  ...        Created Time Course duration Months of study  \
1           Organic  ... 2024-06-21 15:23:00             6.0             NaN   
56       Google Ads  ... 2024-06-20 12:34:00             6.0             NaN   
60              SMM  ... 2024-06-20 11:16:00             6.0             NaN   
65     Facebook Ads  ... 2024-06-20 08:48:00            11.0             NaN   
71     Facebook Ads  ... 2024-06-19 22:31:00            11.0             NaN   
...             ...  ...                 ...             ...             ...   
21574    Google Ads  ... 2023-07-06 04:14:00            11.0             NaN   
21578  Facebook Ads  ... 2023-07-05 18:06:00            11.0             NaN   
21585  Facebook Ads  ... 2023-07-04 11:46:00            11.0             NaN   
21586  Facebook Ads  ... 2023-07-04 10:11:00            11.0            11.0   
21592       Organic  ... 2023-07-03 00:00:00             NaN             NaN   

      Initial Amount Paid  Offer Total Amount         Contact Name  \
1                       0                2000  5805028000056834471   
56                   1000                9000  5805028000056690015   
60                   1000                9000  5805028000056578244   
65                   1000               11000  5805028000056576103   
71                   1

In [ ]:
deals_df['Offer Total Amount'].value_counts()

Offer Total Amount
11000         1812
0              789
11500          380
5000           289
4000           249
3500           129
9000           114
2500            67
2000            62
3000            56
4500            55
€ 2.900,00      20
1200             6
1000             3
10000            2
1500             2
6500             1
1                1
€ 11398,00       1
11111            1
6000             1
Name: count, dtype: int64

In [ ]:
deals_df['Initial Amount Paid'].value_counts()

Initial Amount Paid
1000          2559
0              817
300            185
500             91
350             80
2000            57
11000           35
200             28
11500           24
3500            20
€ 3.500,00      16
450             16
1500            15
5000            14
4000            13
100             12
4500            10
3000            10
400             10
600              3
1200             2
1                1
700              1
9                1
Name: count, dtype: int64

## Обработка столбцов Initial Amount Paid и Offer Total Amount

In [ ]:
deals_df['Initial Amount Paid'] = deals_df['Initial Amount Paid'] \
    .replace(r'[€]', '', regex=True) \
    .replace(r'\s+', '', regex=True) \
    .replace(r'\.', '', regex=True) \
    .replace(r',', '.', regex=True) \
    .astype(float)

deals_df['Offer Total Amount'] = deals_df['Offer Total Amount'] \
    .replace(r'[€]', '', regex=True) \
    .replace(r'\s+', '', regex=True) \
    .replace(r'\.', '', regex=True) \
    .replace(r',', '.', regex=True) \
    .astype(float)

In [ ]:
deals_df['Offer Total Amount'].value_counts()

Offer Total Amount
11000.0    1812
0.0         789
11500.0     380
5000.0      289
4000.0      249
3500.0      129
9000.0      114
2500.0       67
2000.0       62
3000.0       56
4500.0       55
2900.0       20
1200.0        6
1000.0        3
10000.0       2
1500.0        2
6500.0        1
1.0           1
11398.0       1
11111.0       1
6000.0        1
Name: count, dtype: int64

In [ ]:
deals_df['Initial Amount Paid'].value_counts()

Initial Amount Paid
1000.0     2559
0.0         817
300.0       185
500.0        91
350.0        80
2000.0       57
3500.0       36
11000.0      35
200.0        28
11500.0      24
450.0        16
1500.0       15
5000.0       14
4000.0       13
100.0        12
4500.0       10
3000.0       10
400.0        10
600.0         3
1200.0        2
1.0           1
700.0         1
9.0           1
Name: count, dtype: int64

In [ ]:
deals_df['Offer Total Amount'].fillna(0, inplace=True)
deals_df['Initial Amount Paid'].fillna(0, inplace=True)

In [ ]:
deals_df.loc[deals_df['Initial Amount Paid'] > deals_df['Offer Total Amount'], ['Initial Amount Paid', 'Offer Total Amount']] = deals_df.loc[deals_df['Initial Amount Paid'] > deals_df['Offer Total Amount'], ['Offer Total Amount', 'Initial Amount Paid']].values
deals_df

Id Deal Owner Name        Closing Date  \
0      5805028000056864695        Ben Hall                 NaT   
1      5805028000056859489   Ulysses Adams                 NaT   
2      5805028000056832357   Ulysses Adams 2024-06-21 14:45:00   
3      5805028000056824246        Eva Kent 2024-06-21 13:32:00   
4      5805028000056873292        Ben Hall 2024-06-21 13:21:00   
...                    ...             ...                 ...   
21587  5805028000000935025    Kevin Parker 2023-07-06 00:00:00   
21589  5805028000000948010      Jane Smith 2023-08-29 00:00:00   
21590  5805028000000945016      Jane Smith 2023-08-29 00:00:00   
21591  5805028000000927004       Bob Brown 2023-07-09 00:00:00   
21592  5805028000000922001       Bob Brown 2023-07-03 17:03:00   

                 Quality     Stage             Lost Reason  \
0                Unknown  New Lead                 Unknown   
1                Unknown  New Lead                 Unknown   
2         D - Non Target      Lost              Non target   
3      E - Non Qualified      Lost          Invalid number   
4         D - Non Target      Lost              Non target   
...                  ...       ...                     ...   
21587  E - Non Qualified      Lost          Doesn't Answer   
21589         B - Medium      Lost     needs time to think   
21590           A - High      Lost        Changed Decision   
21591     D - Non Target      Lost  Does not speak English   
21592  E - Non Qualified      Lost                 Refugee   

                       Campaign             Content           Term  \
0                 03.07.23women                 v16          women   
1                       Unknown             Unknown        Unknown   
2                    engwien_AT               b1-at     21_06_2024   
3      04.07.23recentlymoved_DE  bloggersvideo14com  recentlymoved   
4                  discovery_DE             website        Unknown   
...                         ...                 ...            ...   
21587           02.07.23wide_DE                  b3           wide   
21589             03.07.23women                  b3          women   
21590           02.07.23wide_DE                  b3           wide   
21591             03.07.23women                  b3          women   
21592                   Unknown             Unknown        Unknown   

               Source  ...        Created Time Course duration  \
0        Facebook Ads  ... 2024-06-21 15:30:00             NaN   
1             Organic  ... 2024-06-21 15:23:00             6.0   
2      Telegram posts  ... 2024-06-21 00:00:00             NaN   
3        Facebook Ads  ... 2024-06-21 00:00:00             NaN   
4          Google Ads  ... 2024-06-21 00:00:00             NaN   
...               ...  ...                 ...             ...   
21587    Facebook Ads  ... 2023-07-04 09:34:00             NaN   
21589    Facebook Ads  ... 2023-07-04 07:10:00             NaN   
21590    Facebook Ads  ... 2023-07-03 20:39:00             NaN   
21591    Facebook Ads  ... 2023-07-03 20:17:00             NaN   
21592         Organic  ... 2023-07-03 00:00:00             NaN   

      Months of study Initial Amount Paid  Offer Total Amount  \
0                 NaN                 0.0                 0.0   
1                 NaN                 0.0              2000.0   
2                 NaN                 0.0                 0.0   
3                 NaN                 0.0                 0.0   
4                 NaN                 0.0                 0.0   
...               ...                 ...                 ...   
21587             NaN                 0.0                 0.0   
21589             NaN                 0.0                 0.0   
21590             NaN                 0.0                 0.0   
21591             NaN                 0.0                 0.0   
21592             NaN                 0.0                 0.0   

              Contact Name  City  SLA Seconds Difference in Days  \
0  

In [ ]:
errors = deals_df.loc[deals_df['Initial Amount Paid'] > deals_df['Offer Total Amount']]
print(errors)

Empty DataFrame
Columns: [Id, Deal Owner Name, Closing Date, Quality, Stage, Lost Reason, Campaign, Content, Term, Source, Payment Type, Product, Education Type, Created Time, Course duration, Months of study, Initial Amount Paid, Offer Total Amount, Contact Name, City, SLA Seconds, Difference in Days, Level of Deutsch_1]
Index: []

[0 rows x 23 columns]


In [ ]:
errors = deals_df.loc[
    (deals_df['Initial Amount Paid'].notna()) &
    (deals_df['Offer Total Amount'].notna()) &
    (deals_df['Initial Amount Paid'] > deals_df['Offer Total Amount'])
]

In [ ]:
fixed_rows = deals_df['Initial Amount Paid'] > deals_df['Offer Total Amount']
deals_df.loc[fixed_rows, ['Initial Amount Paid', 'Offer Total Amount']] = \
    deals_df.loc[fixed_rows, ['Offer Total Amount', 'Initial Amount Paid']].values
print(f"Количество исправленных строк: {fixed_rows.sum()}")

Количество исправленных строк: 0


### Вывод итогов

In [ ]:
deals_df.head()

Id Deal Owner Name        Closing Date            Quality  \
0  5805028000056864695        Ben Hall                 NaT            Unknown   
1  5805028000056859489   Ulysses Adams                 NaT            Unknown   
2  5805028000056832357   Ulysses Adams 2024-06-21 14:45:00     D - Non Target   
3  5805028000056824246        Eva Kent 2024-06-21 13:32:00  E - Non Qualified   
4  5805028000056873292        Ben Hall 2024-06-21 13:21:00     D - Non Target   

      Stage     Lost Reason                  Campaign             Content  \
0  New Lead         Unknown             03.07.23women                 v16   
1  New Lead         Unknown                   Unknown             Unknown   
2      Lost      Non target                engwien_AT               b1-at   
3      Lost  Invalid number  04.07.23recentlymoved_DE  bloggersvideo14com   
4      Lost      Non target              discovery_DE             website   

            Term          Source  ...        Created Time Course duration  \
0          women    Facebook Ads  ... 2024-06-21 15:30:00             NaN   
1        Unknown         Organic  ... 2024-06-21 15:23:00             6.0   
2     21_06_2024  Telegram posts  ... 2024-06-21 00:00:00             NaN   
3  recentlymoved    Facebook Ads  ... 2024-06-21 00:00:00             NaN   
4        Unknown      Google Ads  ... 2024-06-21 00:00:00             NaN   

  Months of study Initial Amount Paid  Offer Total Amount  \
0             NaN                 0.0                 0.0   
1             NaN                 0.0              2000.0   
2             NaN                 0.0                 0.0   
3             NaN                 0.0                 0.0   
4             NaN                 0.0                 0.0   

          Contact Name  City  SLA Seconds Difference in Days  \
0  5805028000056849495  None          NaN                NaN   
1  5805028000056834471  None          NaN                NaN   
2  5805028000056854421  None       1603.0                0.0   
3  5805028000056889351  None       3604.0                0.0   
4  5805028000056876176  None       3192.0                0.0   

  Level of Deutsch_1  
0            Unknown  
1            Unknown  
2            Unknown  
3            Unknown  
4            Unknown  

[5 rows x 23 columns]

In [ ]:
deals_df.describe()

Closing Date                   Created Time  \
count                          13031                          19614   
mean   2024-01-28 12:00:11.220934400  2024-01-27 21:09:00.550627072   
min              2023-07-03 17:03:00            2022-10-11 00:00:00   
25%              2023-11-10 22:00:00            2023-11-18 14:38:00   
50%              2024-02-08 00:00:00            2024-02-06 00:51:30   
75%              2024-04-18 07:01:30            2024-04-14 11:27:15   
max              2024-12-11 00:00:00            2024-06-21 15:30:00   
std                              NaN                            NaN   

       Course duration  Months of study  Initial Amount Paid  \
count      3490.000000       827.000000         19614.000000   
mean         10.196275         5.446191           190.983532   
min           6.000000         0.000000             0.000000   
25%          11.000000         3.000000             0.000000   
50%          11.000000         5.000000             0.000000   
75%          11.000000         8.000000             0.000000   
max          11.000000        11.000000         11000.000000   
std           1.836741         2.928601           709.057172   

       Offer Total Amount   SLA Seconds  Difference in Days  
count        19614.000000  1.481400e+04        13031.000000  
mean          1485.631131  1.120312e+05           15.294145  
min              0.000000  3.000000e+00            0.000000  
25%              0.000000  4.392000e+03            0.000000  
50%              0.000000  1.974900e+04            3.000000  
75%              0.000000  5.581375e+04           12.000000  
max          11500.000000  2.690846e+07          357.000000  
std           3586.243202  7.302550e+05           32.398736

In [ ]:
deals_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19614 entries, 0 to 21592
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   19614 non-null  object        
 1   Deal Owner Name      19614 non-null  object        
 2   Closing Date         13031 non-null  datetime64[ns]
 3   Quality              19614 non-null  object        
 4   Stage                19614 non-null  object        
 5   Lost Reason          19614 non-null  object        
 6   Campaign             19614 non-null  object        
 7   Content              19614 non-null  object        
 8   Term                 19614 non-null  object        
 9   Source               19614 non-null  object        
 10  Payment Type         477 non-null    object        
 11  Product              3494 non-null   object        
 12  Education Type       3215 non-null   object        
 13  Created Time         19614 non-null 

In [ ]:
deals_df.to_excel("Deals_clear.xlsx")

In [ ]:
deals_df.to_csv('Deals_clear.csv', index=False)

In [ ]:
files.download('Deals_clear.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2. Описательная стастика

## `contacts_df`



In [ ]:
contacts_df.describe()

Id Contact Owner Name      Created Time  \
count                 18548              18548             18548   
unique                18548                 28             17921   
top     5805028000056907001      Charlie Davis  10.06.2024 09:00   
freq                      1               2018                13   

           Modified Time  
count              18548  
unique             16580  
top     13.06.2024 17:08  
freq                  25

In [ ]:
print("Описательная статистика количественных полей:")
print("Уникальные значения Id:", contacts_df['Id'].nunique())
print("Created Time диапазон:", contacts_df['Created Time'].min(), "-", contacts_df['Created Time'].max())
print("Modified Time диапазон:", contacts_df['Modified Time'].min(), "-", contacts_df['Modified Time'].max())

Описательная статистика количественных полей:
Уникальные значения Id: 18548
Created Time диапазон: 01.01.2024 02:48 - 31.12.2023 22:15
Modified Time диапазон: 01.01.2024 21:49 - 31.12.2023 00:56


In [ ]:
categorical_fields = ['Contact Owner Name']
for field in categorical_fields:
    print(f"\nАнализ поля: {field}")
    print("Количество уникальных категорий:", contacts_df[field].nunique())
    print("Частота встречаемости категорий:\n", contacts_df[field].value_counts())


Анализ поля: Contact Owner Name
Количество уникальных категорий: 28
Частота встречаемости категорий:
 Contact Owner Name
Charlie Davis      2018
Ulysses Adams      1816
Julia Nelson       1769
Paula Underwood    1487
Quincy Vincent     1416
Nina Scott         1150
Ben Hall           1038
Victor Barnes       967
Cara Iverson        880
Rachel White        782
Jane Smith          754
Bob Brown           685
Ian Miller          684
Diana Evans         678
Yara Edwards        655
Amy Green           621
Eva Kent            365
Kevin Parker        325
Mason Roberts       217
George King         144
Sam Young            37
Alice Johnson        27
Oliver Taylor        19
Zachary Foster        8
Tina Zhang            2
Wendy Clark           2
False                 1
Derek James           1
Name: count, dtype: int64


## Вывод: таблица содержит 18,548 уникальных записей по id, так же содержит 28 уникальных менеджеров. Самые активные контакт-менеджеры: Charlie Davis (2018 записей), Ulysses Adams (1816 записей) и Julia Nelson (1769 записей).
## Рекомендация: оптимизировать нагрузку на контакт-менеджеров с большим количеством записей, перераспределив задачи.

# `calls_df`

In [ ]:
calls_df.describe(include='all')

Id                Call Start Time Call Owner Name  \
count                 88815                          88815           88815   
unique                88815                            NaN              32   
top     5805028000056893619                            NaN    Yara Edwards   
freq                      1                            NaN            8234   
mean                    NaN  2024-02-04 12:52:24.652592384             NaN   
min                     NaN            2023-06-30 09:20:00             NaN   
25%                     NaN            2023-11-23 12:45:00             NaN   
50%                     NaN            2024-02-16 13:23:00             NaN   
75%                     NaN            2024-04-22 12:51:00             NaN   
max                     NaN            2024-06-21 15:30:00             NaN   
std                     NaN                            NaN             NaN   

                  CONTACTID Call Type  Call Duration (in seconds)  \
count                 88815     88815                88815.000000   
unique                15214         3                         NaN   
top     5805028000003329100  Outbound                         NaN   
freq                     94     81146                         NaN   
mean                    NaN       NaN                  171.030108   
min                     NaN       NaN                    0.000000   
25%                     NaN       NaN                    4.000000   
50%                     NaN       NaN                    9.000000   
75%                     NaN       NaN                  108.000000   
max                     NaN       NaN                 7625.000000   
std                     NaN       NaN                  408.379382   

             Call Status Outgoing Call Status  Scheduled in CRM  \
count              88815                88815           88815.0   
unique                 6                    3               3.0   
top     Attended Dialled            Completed               0.0   
freq               67502                81141           81140.0   
mean                 NaN                  NaN               NaN   
min                  NaN                  NaN               NaN   
25%                  NaN                  NaN               NaN   
50%                  NaN                  NaN               NaN   
75%                  NaN                  NaN               NaN   
max                  NaN                  NaN               NaN   
std                  NaN                  NaN               NaN   

        Call Duration (min)  
count          88815.000000  
unique                  NaN  
top                     NaN  
freq                    NaN  
mean               2.850502  
min                0.000000  
25%                0.066667  
50%                0.150000  
75%                1.800000  
max              127.083333  
std                6.806323

In [ ]:
categorical_fields = ['Call Owner Name', 'Call Type', 'Call Status', 'Outgoing Call Status', 'Scheduled in CRM']

for field in categorical_fields:
    counts = calls_df[field].value_counts()
    print(f"Анализ поля '{field}':")
    print(counts)
    print()

Анализ поля 'Call Owner Name':
Call Owner Name
Yara Edwards       8234
Julia Nelson       6942
Ian Miller         6717
Charlie Davis      6580
Diana Evans        6531
Ulysses Adams      5580
Amy Green          5433
Victor Barnes      5209
Nina Scott         5094
Kevin Parker       5050
Paula Underwood    4363
Quincy Vincent     4110
Jane Smith         3400
Cara Iverson       3014
Ben Hall           2776
John Doe           2519
Alice Johnson      1220
Mason Roberts      1122
Derek James         854
George King         805
Zachary Foster      501
Eva Kent            480
Sam Young           445
Rachel White        422
Fiona Jackson       413
Xander Dean         292
Ethan Harris        271
Hannah Lee          173
Wendy Clark         160
Bob Brown            93
Oliver Taylor         8
Tina Zhang            4
Name: count, dtype: int64

Анализ поля 'Call Type':
Call Type
Outbound    81146
Missed       4980
Inbound      2689
Name: count, dtype: int64

Анализ поля 'Call Status':
Call Status
Att

In [ ]:
call_type_status_ct = pd.crosstab(calls_df['Call Type'], calls_df['Outgoing Call Status'])
print("Cross-таблица: Call Type vs Outgoing Call Status")
print(call_type_status_ct)

owner_status_ct = pd.crosstab(calls_df['Call Owner Name'], calls_df['Call Status'])
print("\nCross-таблица: Call Owner Name vs Call Status")
print(owner_status_ct)

Cross-таблица: Call Type vs Outgoing Call Status
Outgoing Call Status  Completed  Overdue  Unknown
Call Type                                        
Inbound                       0        0     2689
Missed                        0        0     4980
Outbound                  81141        5        0

Cross-таблица: Call Owner Name vs Call Status
Call Status      Attended Dialled  Missed  Overdue  Received  \
Call Owner Name                                                
Alice Johnson                 909       3        0        13   
Amy Green                    3823     248        0       138   
Ben Hall                     2415       0        0         0   
Bob Brown                      89       0        0         0   
Cara Iverson                 2245       0        0         0   
Charlie Davis                5016     546        0       235   
Derek James                   684       0        0        81   
Diana Evans                  5025     287        0       249   
Ethan Harris  

In [ ]:
for field in categorical_fields:
    avg_duration = calls_df.groupby(field)['Call Duration (min)'].mean()
    print(f"\nСредняя продолжительность звонков для каждой категории в '{field}':")
    print(avg_duration)


Средняя продолжительность звонков для каждой категории в 'Call Owner Name':
Call Owner Name
Alice Johnson      1.892418
Amy Green          1.722823
Ben Hall           4.262194
Bob Brown          4.653047
Cara Iverson       2.818464
Charlie Davis      3.758419
Derek James        3.376561
Diana Evans        3.116394
Ethan Harris       0.852583
Eva Kent           6.762604
Fiona Jackson      0.508313
George King        3.208323
Hannah Lee         0.994990
Ian Miller         2.001335
Jane Smith         3.506760
John Doe           2.264748
Julia Nelson       3.456117
Kevin Parker       2.114119
Mason Roberts      4.006581
Nina Scott         2.208402
Oliver Taylor      2.139583
Paula Underwood    3.371568
Quincy Vincent     4.693443
Rachel White       2.533570
Sam Young          5.786404
Tina Zhang         0.000000
Ulysses Adams      3.482470
Victor Barnes      2.395319
Wendy Clark        1.120417
Xander Dean        1.777626
Yara Edwards       1.674648
Zachary Foster     1.571457
Name: Call 

## Вывод: таблица содержит 88,815 записей, Основной тип — исходящие звонки (Outbound, 81,146), что составляет подавляющее большинство. Пропущенные звонки (Missed) составляют 4,980, а входящие звонки (Inbound) — 2,689.Большая часть звонков завершена успешно (Completed, 81,141).Большинство звонков (81,140) не были запланированы в CRM.Наибольшее количество звонков зафиксировано у Yara Edwards (8,234), Julia Nelson (6,942) и Ian Miller (6,717). У ряда сотрудников звонков очень мало, например Tina Zhang (4) и Oliver Taylor (8).
## Рекомендация: увеличить внимание на пропущенные звонки и их причины для повышения успешности.Оптимизировать распределение звонков между менеджерами с учётом их нагрузок.Анализ входящих звонков из-за их большей средней продолжительности, что может указывать на важность этих контактов.

# `spend_df`

In [ ]:
print("Описательная статистика количественных данных:")
print(spend_df[['Impressions', 'Spend', 'Clicks']].describe())

Описательная статистика количественных данных:
         Impressions         Spend        Clicks
count   14887.000000  14887.000000  14887.000000
mean     3427.332774      9.983453     31.062806
std     13395.468506     31.159348     98.414714
min         0.000000      0.000000      0.000000
25%        34.000000      0.300000      0.000000
50%       286.000000      2.330000      4.000000
75%      1060.000000      8.885000     17.000000
max    431445.000000    774.000000   2415.000000


In [ ]:
print("Диапазон дат:")
print(f"{spend_df['Date'].min()} - {spend_df['Date'].max()}")
print("Количество уникальных дней:")
print(spend_df['Date'].nunique())

Диапазон дат:
2023-07-03 00:00:00 - 2024-06-21 00:00:00
Количество уникальных дней:
355


In [ ]:
categorical_fields = ['Source']

for field in categorical_fields:
    print(f"Анализ столбца '{field}':")
    print("Количество уникальных категорий:", spend_df[field].nunique())
    print("Частота категорий:")
    print(spend_df[field].value_counts())
    print()

Анализ столбца 'Source':
Количество уникальных категорий: 13
Частота категорий:
Source
Facebook Ads      8749
Tiktok Ads        2242
Youtube Ads       1424
Google Ads         912
Telegram posts     469
Webinar            321
Bloggers           259
Organic            231
SMM                176
CRM                 79
Partnership         15
Offline              8
Radio                2
Name: count, dtype: int64



## Вывод: среднее количество показов (Impressions) — 3,427, но высокое стандартное отклонение (13,395) указывает на значительные различия в данных.Средние затраты (Spend) — около 10, но также значительное отклонение (31.15), с максимумом 774.Среднее количество кликов (Clicks) — около 31, с максимумом 2,415.Данные охватывают 355 уникальных дней с 3 июля 2023 года по 21 июня 2024 года.Наиболее популярный источник — Facebook Ads (8,749 записей, около 59% всех данных).За ним следуют Tiktok Ads (2,242) и Youtube Ads (1,424).
## Рекомендации: Увеличить вложения в Youtube Ads, если эти кампании имеют стабильные показатели конверсии при меньших затратах.Удалить или оптимизировать слабые каналы, такие как Radio и Offline, если они не приносят результатов.

# `deals_df`

In [ ]:
categorical_fields = ['Quality', 'Stage', 'Source', 'Product']

for field in categorical_fields:
    print(f"Описательная статистика для {field}:")
    print(deals_df[field].describe())
    print("\n")

Описательная статистика для Quality:
count                 19614
unique                    6
top       E - Non Qualified
freq                   6100
Name: Quality, dtype: object


Описательная статистика для Stage:
count     19614
unique       13
top        Lost
freq      13890
Name: Stage, dtype: object


Описательная статистика для Source:
count            19614
unique              12
top       Facebook Ads
freq              4727
Name: Source, dtype: object


Описательная статистика для Product:
count                  3494
unique                    5
top       Digital Marketing
freq                   1933
Name: Product, dtype: object




In [ ]:
for field in categorical_fields:
    print(f"Распределение категорий для {field}:")
    print(deals_df[field].value_counts())
    print("\n")

Распределение категорий для Quality:
Quality
E - Non Qualified    6100
D - Non Target       5972
C - Low              3373
Unknown              2230
B - Medium           1525
A - High              414
Name: count, dtype: int64


Распределение категорий для Stage:
Stage
Lost                         13890
Call Delayed                  2163
Registered on Webinar         2065
Payment Done                   843
Waiting For Payment            323
Qualificated                   128
Registered on Offline Day       85
Need to Call - Sales            32
Need To Call                    31
Test Sent                       25
Need a consultation             23
New Lead                         5
Free Education                   1
Name: count, dtype: int64


Распределение категорий для Source:
Source
Facebook Ads      4727
Google Ads        4113
Tiktok Ads        2003
SMM               1668
Youtube Ads       1618
Organic           1469
CRM               1454
Bloggers          1073
Telegram posts     9

In [ ]:
cross_table_1 = pd.crosstab(deals_df['Quality'], deals_df['Stage'])
print("Cross-таблица: Quality vs Stage")
print(cross_table_1)

cross_table_2 = pd.crosstab(deals_df['Source'], deals_df['Product'])
print("\nCross-таблица: Source vs Product")
print(cross_table_2)

Cross-таблица: Quality vs Stage
Stage              Call Delayed  Free Education  Lost  Need To Call  \
Quality                                                               
A - High                     24               0   219             0   
B - Medium                   88               0   978             0   
C - Low                     306               1  2488             0   
D - Non Target             1728               0  4128             0   
E - Non Qualified            17               0  6076             0   
Unknown                       0               0     1            31   

Stage              Need a consultation  Need to Call - Sales  New Lead  \
Quality                                                                  
A - High                             0                     0         0   
B - Medium                           0                     1         0   
C - Low                              0                     2         0   
D - Non Target               

In [ ]:
for field in categorical_fields:
    avg_payment = deals_df.groupby(field)['Initial Amount Paid'].mean()
    print(f"\nСредняя сумма начальных платежей для каждой категории в {field}:")
    print(avg_payment)


Средняя сумма начальных платежей для каждой категории в Quality:
Quality
A - High             886.352657
B - Medium           719.934426
C - Low              492.588200
D - Non Target        98.057770
E - Non Qualified      4.754098
Unknown                2.242152
Name: Initial Amount Paid, dtype: float64

Средняя сумма начальных платежей для каждой категории в Stage:
Stage
Call Delayed                  178.848821
Free Education                300.000000
Lost                          136.785457
Need To Call                    0.000000
Need a consultation             0.000000
Need to Call - Sales            0.000000
New Lead                        0.000000
Payment Done                 1116.193357
Qualificated                  156.250000
Registered on Offline Day      58.823529
Registered on Webinar           0.048426
Test Sent                     240.000000
Waiting For Payment          1507.120743
Name: Initial Amount Paid, dtype: float64

Средняя сумма начальных платежей для каждой ка

In [ ]:
for field in categorical_fields:
    avg_offer = deals_df.groupby(field)['Offer Total Amount'].mean()
    print(f"\nСредняя сумма Offer Total Amount для каждой категории в {field}:")
    print(avg_offer)


Средняя сумма Offer Total Amount для каждой категории в Quality:
Quality
A - High             6762.801932
B - Medium           5937.967213
C - Low              3621.686333
D - Non Target        795.800569
E - Non Qualified      50.573770
Unknown                 3.139013
Name: Offer Total Amount, dtype: float64

Средняя сумма Offer Total Amount для каждой категории в Stage:
Stage
Call Delayed                  1530.286177
Free Education                4000.000000
Lost                          1162.164075
Need To Call                     0.000000
Need a consultation              0.000000
Need to Call - Sales             0.000000
New Lead                       400.000000
Payment Done                  7423.014235
Qualificated                   810.156250
Registered on Offline Day       58.823529
Registered on Webinar            1.937046
Test Sent                     2560.000000
Waiting For Payment          10050.773994
Name: Offer Total Amount, dtype: float64

Средняя сумма Offer Total Amo

## Вывод: большинство лидов относятся к категориям E - Non Qualified (6,100) и D - Non Target (5,972).Высококачественных лидов (A - High) всего 414.Начальная сумма платежей и общий объем предложения прямо пропорциональны качеству, где лиды категории "A" имеют самый высокий средний платеж (886.35) и объем предложения (6,762.80).Основной этап — Lost (13,890), что указывает на высокую долю потерянных сделок.Успешные сделки (Payment Done) составляют 843 записи, с самым высоким средним предложением (7,423.01) и начальной суммой платежей (1,116.19).Лидером по количеству лидов является Facebook Ads (4,727), за ним следуют Google Ads (4,113) и Tiktok Ads (2,003).Источник Offline имеет всего 2 записи, но с самым высоким средним объемом предложения (5,500.00).Organic показывает сильный потенциал с высоким средним предложением (2,596.26) и платежами (365.42).Этапы Waiting For Payment имеют самое высокое среднее предложение (10,050.77), указывая на высокий потенциал завершения.Самый популярный продукт — Digital Marketing (1,933 лидов), за ним следуют UX/UI Design (996) и Web Developer (561).UX/UI Design имеет самые высокие средние начальные платежи (1,123.34) и объем предложения (8,712.09), что делает его лидером по общей стоимости.


## Рекомендации: сфокусироваться на привлечении высококачественных лидов, сокращая долю "Non Qualified" и "Non Target".Пересмотреть источники рекламы с низкой конверсией для улучшения качества.Активнее работать с этапами "Waiting For Payment" и "Registered on Webinar" для увеличения успешных завершений.Увеличить инвестиции в Organic и Webinar, которые демонстрируют высокую отдачу.Минимизировать затраты на малозначимые каналы, такие как Offline, если они не приносят ощутимых результатов.Развивать предложения UX/UI Design и Digital Marketing, как наиболее успешных и популярных продуктов.Рассмотреть пересмотр или оптимизацию продуктов с минимальной активностью, таких как Find yourself in IT и Data Analytics.